<a href="https://colab.research.google.com/github/VardhanKirti/Pharma-Bot/blob/main/medicine_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/content/final_cleaned_MID.csv')

In [4]:
df.head()

,Name,Contains,ProductIntroduction,ProductUses,ProductBenefits,SideEffect,HowToUse,HowWorks,QuickTips,SafetyAdvice,Chemical_Class,Habit_Forming,Therapeutic_Class,Action_Class
0,Andol 0.5mg Tablet,Haloperidol (0.5mg),Andol 0.5mg Tablet can also be used for treati...,Schizophrenia,Schizophrenia Schizophrenia is a mental disord...,Most side effects do not require any medical a...,Take this medicine in the dose and duration as...,Andol 0.5mg Tablet is a typical antipsychotic....,Andol 0.5mg Tablet helps treat schizophrenia. ...,Alcohol Andol 0.5mg Tablet may cause excessive...,Butyrophenone Derivative,False,NEURO CNS,Typical Antipsychotic
1,Avastin 100mg Injection,Bevacizumab (100mg),Avastin 100mg Injection is an effective medici...,Cancer Of Colon And Rectum Non-Small Cell Lung...,Cancer of colon and rectum Cancer of the colon...,Most side effects do not require any medical a...,Your doctor or nurse will give you this medici...,Avastin 100mg Injection is a anti-angiogenic m...,"Avastin 100mg Injection is an effective, first...",Alcohol It is not known whether it is safe to ...,Monoclonal antibody (mAb),False,ANTI NEOPLASTICS,Vascular endothelial growth factor (VEGF) inhi...
2,Actorise 40 Injection,Darbepoetin alfa (40mcg),Actorise 40 Injection is a medicine that needs...,Anemia Due To Chronic Kidney Disease Treatment...,Treatment of Anemia due to chronic kidney dise...,Most side effects do not require any medical a...,Your doctor or nurse will give you this medici...,Actorise 40 Injection is an erythropoiesis-sti...,Actorise 40 Injection helps in the treatment o...,Alcohol It is not known whether it is safe to ...,"Amino Acids, Peptides Analogues",False,BLOOD RELATED,Erythropoiesis-stimulating agent (ESA
3,Actorise 25 Injection,Darbepoetin alfa (25mcg),Actorise 25 Injection is a medicine that needs...,Anemia Due To Chronic Kidney Disease Treatment...,Treatment of Anemia due to chronic kidney dise...,Most side effects do not require any medical a...,Your doctor or nurse will give you this medici...,Actorise 25 Injection is an erythropoiesis-sti...,Actorise 25 Injection helps in the treatment o...,Alcohol It is not known whether it is safe to ...,"Amino Acids, Peptides Analogues",False,BLOOD RELATED,Erythropoiesis-stimulating agent (ESA
4,Actorise 60 Injection,Darbepoetin alfa (60mcg),Actorise 60 Injection is a medicine that needs...,Anemia Due To Chronic Kidney Disease Treatment...,Treatment of Anemia due to chronic kidney dise...,Most side effects do not require any medical a...,Your doctor or nurse will give you this medici...,Actorise 60 Injection is an erythropoiesis-sti...,Actorise 60 Injection helps in the treatment o...,Alcohol It is not known whether it is safe to ...,"Amino Acids, Peptides Analogues",False,BLOOD RELATED,Erythropoiesis-stimulating agent (ESA


In [5]:
df.shape

(148272, 14)

In [6]:
df.columns

Index(['Name', 'Contains', 'ProductIntroduction', 'ProductUses',
       'ProductBenefits', 'SideEffect', 'HowToUse', 'HowWorks', 'QuickTips',
       'SafetyAdvice', 'Chemical_Class', 'Habit_Forming', 'Therapeutic_Class',
       'Action_Class'],
      dtype='object')

In [16]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553318 sha256=01892e597ef40e5c4831323bbe263b4fcfa2949966244bde4636999f0fa79bdd
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [17]:

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from annoy import AnnoyIndex

In [18]:
try:

    df = pd.read_csv('final_cleaned_MID.csv')
    df.fillna('', inplace=True)

    df['Name'] = df['Name'].str.strip()
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: The file 'final_reduced_MID.csv' was not found.")

Data loaded successfully.


In [19]:

df['corpus'] = (
    df['ProductUses'].astype(str) + ' ' +
    df['ProductBenefits'].astype(str) + ' ' +
    df['HowWorks'].astype(str) + ' ' +
    (df['Therapeutic_Class'].astype(str) + ' ') * 3
)

print("Building the TF-IDF model...")
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['corpus'])
print("TF-IDF model built.")

Building the TF-IDF model...
TF-IDF model built.


In [20]:

num_features = tfidf_matrix.shape[1]
annoy_index = AnnoyIndex(num_features, 'angular')

for i in range(tfidf_matrix.shape[0]):
    vector = tfidf_matrix[i].toarray()[0]
    annoy_index.add_item(i, vector)

annoy_index.build(10)


True

In [25]:

indices = pd.Series(df.index, index=df['Name'])

def get_recommendations(name, annoy_index, df, indices, top_n=5):
    """
    Gets recommendations instantly using the pre-built Annoy index.
    """
    try:
        idx = indices[name]


        if isinstance(idx, pd.Series):
            idx = idx.iloc[0]

        neighbor_indices = annoy_index.get_nns_by_item(idx, top_n + 1)

        medicine_indices = [i for i in neighbor_indices if i != idx][:top_n]

        return df['Name'].iloc[medicine_indices].tolist()
    except (IndexError, KeyError):
        return []





In [ ]:
# --- Q&A Creation ---
qa_pairs = []

for index, row in df.iterrows():

    if (index + 1) % 1000 == 0:
        print(f"  Processing row {index + 1} of {len(df)}...")

    name = row['Name']


     # Recommendations
    recommendations = get_recommendations(name, annoy_index, df, indices)
    if recommendations:
        rec_list = ", ".join(recommendations)
        qa_pairs.append({
            'Question': f"What is an alternative to {name}?",
            'Answer': f"Some other medicines used for similar conditions include: {rec_list}. However, you must consult your doctor before making any changes to your treatment."
        })
        qa_pairs.append({
            'Question': f"Are there other options besides {name}?",
            'Answer': f"Medicines like {rec_list} also treat similar issues. Please speak with a healthcare professional to see if they are right for you."
        })

for index, row in df.iterrows():
    name = row['Name']
    uses = row['ProductUses']
    benefits = row['ProductBenefits']
    side_effects = row['SideEffect']
    how_to_use = row['HowToUse']
    how_it_works = row['HowWorks']
    quick_tips = row['QuickTips']
    safety_advice = row['SafetyAdvice']

    # Uses & Benefits
    if benefits != 'No information available.':
        qa_pairs.append({'Question': f"What is {name} used for?", 'Answer': benefits})
        qa_pairs.append({'Question': f"What are the benefits of taking {name}?", 'Answer': benefits})
        qa_pairs.append({'Question': f"Why would a doctor prescribe {name}?", 'Answer': benefits})
        if uses != 'No information available.':
            qa_pairs.append({'Question': f"My doctor said I have {uses.lower()}, will {name} help?", 'Answer': benefits})

    # Side Effects
    if side_effects != 'No information available.':
        qa_pairs.append({'Question': f"What are the side effects of {name}?", 'Answer': side_effects})
        qa_pairs.append({'Question': f"Are there any common side effects for {name}?", 'Answer': side_effects})
        qa_pairs.append({'Question': f"Should I be worried about any side effects with {name}?", 'Answer': side_effects})

    # How to Use
    if how_to_use != 'No information available.':
        qa_pairs.append({'Question': f"How should I take {name}?", 'Answer': how_to_use})
        qa_pairs.append({'Question': f"What is the correct way to use {name}?", 'Answer': how_to_use})
        qa_pairs.append({'Question': f"Can you explain the dosage instructions for {name}?", 'Answer': how_to_use})

    # How it Works
    if how_it_works != 'No information available.':
        qa_pairs.append({'Question': f"How does {name} work?", 'Answer': how_it_works})
        qa_pairs.append({'Question': f"What is the mechanism of action for {name}?", 'Answer': how_it_works})
        qa_pairs.append({'Question': f"Can you explain the science behind how {name} functions?", 'Answer': how_it_works})

    # Quick Tips
    if quick_tips != 'No information available.':
        qa_pairs.append({'Question': f"Do you have any quick tips for someone taking {name}?", 'Answer': quick_tips})
        qa_pairs.append({'Question': f"What are some helpful tips for using {name}?", 'Answer': quick_tips})

    # Safety Advice
    if safety_advice != 'No information available.':
        qa_pairs.append({'Question': f"What is the general safety advice for {name}?", 'Answer': safety_advice})
        alcohol_info = re.search(r'Alcohol\s*(.*?)\s*Pregnancy', safety_advice, re.IGNORECASE | re.DOTALL)
        if alcohol_info:
            qa_pairs.append({'Question': f"Is it safe to drink alcohol with {name}?", 'Answer': alcohol_info.group(1).strip()})
        pregnancy_info = re.search(r'Pregnancy\s*(.*?)\s*Breast feeding', safety_advice, re.IGNORECASE | re.DOTALL)
        if pregnancy_info:
            qa_pairs.append({'Question': f"Can I take {name} if I am pregnant?", 'Answer': pregnancy_info.group(1).strip()})
        driving_info = re.search(r'Driving\s*(.*?)\s*Kidney', safety_advice, re.IGNORECASE | re.DOTALL)
        if driving_info:
            qa_pairs.append({'Question': f"Is it safe to drive while taking {name}?", 'Answer': driving_info.group(1).strip()})



qa_df = pd.DataFrame(qa_pairs).drop_duplicates()




In [27]:

def ask(question):
    results = qa_df[qa_df['Question'].str.lower().str.contains(question.lower())]
    if not results.empty:
        for _, row in results.iterrows():
            print(f"Q: {row['Question']}\nA: {row['Answer']}\n")
    else:
        print("No matching Q&A found.")



In [28]:
ask("side effects of aspirin")

Q: What are the side effects of Aspirin 300mg Tablet?
A: Most side effects do not require any medical attention and disappear as your body adjusts to the medicine. Consult your doctor if they persist or if you’re worried about them Common side effects of Aspirin Heartburn Increased bleeding tendency Nausea Upset stomach Vomiting

Q: What are the side effects of Aspirin 50mg Tablet DR?
A: Most side effects do not require any medical attention and disappear as your body adjusts to the medicine. Consult your doctor if they persist or if you’re worried about them Common side effects of Aspirin Heartburn Increased bleeding tendency Nausea Upset stomach Vomiting



In [29]:
ask("What is an alternative to Paracetamol?")
ask("Are there other options besides Ibuprofen?")

Q: What is an alternative to Paracetamol 120mg Oral Suspension?
A: Some other medicines used for similar conditions include: Brumol 120mg Oral Suspension, Fevstop 120mg Oral Suspension, Parasuit 120mg Oral Suspension, Brikamol Oral Suspension, Fenil Oral Suspension. However, you must consult your doctor before making any changes to your treatment.

Q: What is an alternative to Paracetamol O Tablet?
A: Some other medicines used for similar conditions include: Algina Tablet, Colsun Tablet, Gempar 1K Tablet, Ikopar Tablet, Metacin Tablet. However, you must consult your doctor before making any changes to your treatment.

No matching Q&A found.


In [30]:
ask("Do you have any quick tips for someone taking Insulin?")

Q: Do you have any quick tips for someone taking Insulin 40IU/ml Injection?
A: Hypoglycemia (low blood sugar level) is a common side effect. Hence, regular monitoring of blood sugar levels is required while taking this medicine. Injection below the skin of the abdomen results in faster absorption than other injection sites. Injection sites must be rotated to prevent hard lumps from developing at one site. Opened vials/cartridge are good at room temperature for up to 4 weeks, while unopened vials must be placed in the refrigerator (2°C–8°C).



In [31]:
OUTPUT_FILE = 'chatbot_qa_pairs.csv'
qa_df.to_csv(OUTPUT_FILE, index=False)
print(f"Q&A pairs saved to {OUTPUT_FILE}")

Q&A pairs saved to chatbot_qa_pairs.csv


In [ ]:

import re

def create_qa_pairs(source_filename, output_filename):

    try:
        print(f"Loading data from '{source_filename}'...")
        df = pd.read_csv(source_filename)
        # Fill any potential missing values with a placeholder to prevent errors
        df.fillna('No information available.', inplace=True)
    except FileNotFoundError:
        print(f"Error: The file '{source_filename}' was not found.")
        return

    qa_pairs = []

    print("Generating Q&A pairs for each medicine...")
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        name = row['Name']
        uses = row['ProductUses']
        benefits = row['ProductBenefits']
        side_effects = row['SideEffect']
        how_to_use = row['HowToUse']
        how_it_works = row['HowWorks']
        quick_tips = row['QuickTips']
        safety_advice = row['SafetyAdvice']

        # --- Generate questions based on different columns and templates ---

        # 1. Product Uses & Benefits
        if benefits != 'No information available.':
            qa_pairs.append({'Question': f"What is {name} used for?", 'Answer': benefits})
            qa_pairs.append({'Question': f"What are the benefits of taking {name}?", 'Answer': benefits})
            qa_pairs.append({'Question': f"Why would a doctor prescribe {name}?", 'Answer': benefits})
            if uses != 'No information available.':
                 qa_pairs.append({'Question': f"My doctor said I have {uses.lower()}, will {name} help?", 'Answer': benefits})


        # 2. Side Effects
        if side_effects != 'No information available.':
            qa_pairs.append({'Question': f"What are the side effects of {name}?", 'Answer': side_effects})
            qa_pairs.append({'Question': f"Are there any common side effects for {name}?", 'Answer': side_effects})
            qa_pairs.append({'Question': f"Should I be worried about any side effects with {name}?", 'Answer': side_effects})

        # 3. How to Use
        if how_to_use != 'No information available.':
            qa_pairs.append({'Question': f"How should I take {name}?", 'Answer': how_to_use})
            qa_pairs.append({'Question': f"What is the correct way to use {name}?", 'Answer': how_to_use})
            qa_pairs.append({'Question': f"Can you explain the dosage instructions for {name}?", 'Answer': how_to_use})

        # 4. How it Works
        if how_it_works != 'No information available.':
            qa_pairs.append({'Question': f"How does {name} work?", 'Answer': how_it_works})
            qa_pairs.append({'Question': f"What is the mechanism of action for {name}?", 'Answer': how_it_works})
            qa_pairs.append({'Question': f"Can you explain the science behind how {name} functions?", 'Answer': how_it_works})

        # 5. Quick Tips
        if quick_tips != 'No information available.':
            qa_pairs.append({'Question': f"Do you have any quick tips for someone taking {name}?", 'Answer': quick_tips})
            qa_pairs.append({'Question': f"What are some helpful tips for using {name}?", 'Answer': quick_tips})

        # 6. Safety Advice (with specific sub-questions)
        if safety_advice != 'No information available.':
            qa_pairs.append({'Question': f"What is the general safety advice for {name}?", 'Answer': safety_advice})
            # Use regex to find specific advice for common queries
            alcohol_info = re.search(r'Alcohol\s*(.*?)\s*Pregnancy', safety_advice, re.IGNORECASE | re.DOTALL)
            if alcohol_info:
                qa_pairs.append({'Question': f"Is it safe to drink alcohol with {name}?", 'Answer': alcohol_info.group(1).strip()})

            pregnancy_info = re.search(r'Pregnancy\s*(.*?)\s*Breast feeding', safety_advice, re.IGNORECASE | re.DOTALL)
            if pregnancy_info:
                 qa_pairs.append({'Question': f"Can I take {name} if I am pregnant?", 'Answer': pregnancy_info.group(1).strip()})

            driving_info = re.search(r'Driving\s*(.*?)\s*Kidney', safety_advice, re.IGNORECASE | re.DOTALL)
            if driving_info:
                 qa_pairs.append({'Question': f"Is it safe to drive while taking {name}?", 'Answer': driving_info.group(1).strip()})


    # --- Create a new DataFrame and save to CSV ---
    qa_df = pd.DataFrame(qa_pairs)

    # Drop any duplicate Q&A pairs that might have been generated
    qa_df.drop_duplicates(inplace=True)

    print(f"\nGenerated a total of {len(qa_df)} unique Q&A pairs.")

    try:
        qa_df.to_csv(output_filename, index=False)
        print(f"Successfully saved the Q&A pairs to '{output_filename}'.")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")


# --- HOW TO USE ---
# 1. Set the name of your final cleaned input file.
INPUT_FILE = 'final_reduced_MID.csv'

# 2. Set the desired name for the Q&A output file.
OUTPUT_FILE = 'chatbot_qa_pairs.csv'

# 3. Run the function.
create_qa_pairs(INPUT_FILE, OUTPUT_FILE)
